In [ ]:
# Overview of geospatial data, coordinate systems and geometry types.  

In [53]:
from sedona.spark import *
from pyspark.sql.functions import col, when, expr
from sedona.sql.st_functions import ST_IsValid, ST_IsValidReason, ST_MakeValid
from pyspark.sql import DataFrame

In [3]:
config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Intoduction to a Sedona DataFrame

A Sedona DataFrame is an extension of the standard Spark DataFrame that adds native support for geospatial data types — namely Vector and Raster.

- Vector data includes geometries such as points, lines, and polygons.
- Raster represents gridded or image-based data such as satellite imagery or elevation tiles.

Because these are built-in (native) data types, Sedona and Wherobots know how to handle them automatically.
This means you can:
- Run spatial and raster functions (like `ST_Contains`, `ST_Intersection`, `RS_Clip`, `RS_ZonalStats`) directly in your queries.
- Combine vector and raster data in the same workflow — for example, clipping imagery to a city boundary.
- Scale these operations easily across large datasets.

In [ ]:
# Vector drivers  - Pranav

# GeoJSON
# CSV
# Shapefile

# Show others in comments

## Loading vector data into a Sedona DataFrame

Now that we know what a Sedona DataFrame is, let’s see how to load vector data into one.

In this section, we’ll cover:
- GeoParquet — the most common cloud-native geospatial format
- GeoJSON — flexible and human-readable
- CSV — raw text with WKT/WKB geometries
- Shapefile — the classic desktop GIS format



### GeoParquet

GeoParquet is the preferred format for storing and sharing vector data in the cloud.

It extends the standard Apache Parquet format with a small block of “geo” metadata that describes:
- which column contains the geometry,
- the geometry type (Point, Polygon, etc.),
- its coordinate reference system (CRS), and
- the bounding box of each geometry column.

Because GeoParquet is columnar, compressed, and splittable, it’s ideal for large-scale analytics on cloud object storage like S3.
It also supports spatial predicate push-down — meaning Wherobots can automatically skip reading irrelevant files and row-groups when performing spatial range queries.

In [ ]:
geoparquet_path = "s3://wherobots-examples/data/nyc_buildings.parquet"

df = sedona.read.format("geoparquet").load(geoparquet_path)
df.show(5)

### GeoJSON

Wherobots supports reading GeoJSON files directly using the geojson data source.

This reader understands most GeoJSON variations, including:
- Standard Feature and FeatureCollection objects
- SpatioTemporal Asset Catalog (STAC) files
- GeoJSON files that span multiple lines for readability

When loaded, Wherobots automatically parses the geometry field into its internal Geometry type.

In [ ]:
geojson_multi_path = "s3://wherobots-examples/data/noaa/storms/"

df = sedona.read.format("geojson")\
        .option("multiLine", "true")\
        .load(geojson_multi_path)\
        .selectExpr("explode(features) as features")\
        .select("features.*")

df.show(5)

### Shapefile

The Shapefile format has been around since the early days of GIS and is still used widely in desktop and government datasets.

Wherobots can load Shapefiles directly into a Sedona DataFrame using the shapefile data source.
This works whether you point to a single .shp file or to a directory containing multiple shapefiles.

Wherobots automatically reads the related files (.dbf, .shx, etc.) and converts the geometry column into a native Geometry type.

In [ ]:
shp_path = "s3://wherobots-examples/data/austin_boundaries/"

df = sedona.read.format("shapefile").load(shp_path)
df.show(5)

When the input path is a directory, all shapefiles directly inside that directory will be loaded.
To include shapefiles in subdirectories, set recursiveFileLookup to true:

In [ ]:
df = sedona.read.format("shapefile") \
    .option("recursiveFileLookup", "true") \
    .load("s3://wherobots-examples/data/examples/Global_Landslide_Catalog/")

df.show(5)

In [ ]:
# Raster - Pranav

# Raster

# Show others

## Loading raster data into a Sedona DataFrame

Wherobots can load rasters as native raster columns, allowing you to tile, clip, resample, and compute statistics directly in Spark - just like you would with tabular data.

We’ll cover:
- Reading GeoTIFFs (COGs recommended)
- Reading from STAC APIs and collections

### GeoTIFFs

GeoTIFFs are raster image files that store both pixel values and geospatial metadata (like coordinate reference systems).
A Cloud-Optimized GeoTIFF (COG) is a GeoTIFF structured for fast, partial reads in cloud storage — ideal for distributed systems like Wherobots.

Wherobots' raster reader can load these directly into a Sedona DataFrame.
Each raster becomes one or more tiles, stored in a column of type raster.

In [ ]:
geotiff_path = "s3://wherobots-examples/data/ghs_population/GHS_POP_E1975_GLOBE_R2023A_4326_3ss_V1_0.tif"

df = sedona.read.format("raster").load(geotiff_path)
df.show(5)

Each row represents a raster tile, and by default the raster reader automatically:
- Splits large rasters into tiles.
- Adds x and y columns to indicate each tile’s position.
- Reads file-level metadata (CRS, extent, etc.) internally.

To fine-tune the tiling behavior, you can specify options:

In [ ]:
df = sedona.read.format("raster") \
    .option("tileWidth", "512") \
    .option("tileHeight", "512") \
    .option("retile", "true") \
    .load(geotiff_path)

df.show(5)

> Tip - Use Cloud-Optimized GeoTIFFs (COGs) when possible — they’re optimized for partial reads in cloud storage, which makes distributed processing far more efficient.

### SpatioTemporal Asset Catalog (STAC)

A SpatioTemporal Asset Catalog (STAC) is a standard for describing and organizing geospatial assets — such as satellite imagery, aerial photos, and elevation data — across space and time.

Wherobots includes a built-in STAC Reader, which allows you to load STAC items and collections directly into a Sedona DataFrame. This gives you seamless access to large imagery archives hosted on cloud platforms like AWS, Planetary Computer, or Element84 — all without leaving your Spark environment.

The STAC Reader supports:
- Direct integration with HTTP, HTTPS, S3, or local STAC JSON sources.
- Unified geospatial analysis, so you can join imagery metadata with vector or raster datasets inside Spark.
- Spatial and temporal filter pushdown, meaning filters like `ST_Intersects` or datetime BETWEEN are pushed down to the STAC API, minimizing data transfer and improving query performance.
- Flexible configuration options for partitioning, request limits, and parallel loading — making it scalable for both exploration and production workflows.

You can connect to STAC sources via an HTTPS endpoint, an S3 path, or a local JSON file.

In [ ]:
stac_df = sedona.read.format("stac").load(
    "https://earth-search.aws.element84.com/v1/collections/sentinel-2-pre-c1-l2a"
)

stac_df.printSchema()
stac_df.select("id", "datetime", "geometry", "collection").show(5, truncate=False)

You can control how many items to load, how requests are batched, and how partitions are distributed.

In [ ]:
df = sedona.read.format("stac") \
            .option("itemsLimitMax", "1000")\
            .option("itemsLimitPerRequest", "50")\
            .option("itemsLoadProcessReportThreshold", "500000")\
            .load("https://earth-search.aws.element84.com/v1/collections/sentinel-2-pre-c1-l2a")

In [ ]:
# Intro to Iceberg - Pranav

## Introduction to Managing Spatial Tables with Iceberg

Wherobots extends Apache Iceberg — a modern open table format — to natively support spatial data.
This allows you to manage vector and raster datasets just like any other analytical table, with the same reliability, scalability, and query optimization benefits.

For a data engineer, this means you can use familiar tools (SQL, Spark, Sedona) while gaining spatial awareness at the table level — no need for custom file handling or geospatial indexing setups.

### Why Iceberg Matters for Spatial Data

Iceberg brings all the essentials of modern data lake management — schema evolution, ACID transactions, partition pruning, and time travel.
Wherobots builds on this foundation to add spatial intelligence directly into the table layer.

1. Native Spatial Columns

    Geometry and raster columns are first-class types — not just blobs or strings.

    This means:
    - You can save Sedona DataFrames with geometry or raster columns directly to Iceberg tables.
    - Query them with familiar functions like `ST_Intersects`, `ST_Within`, or `RS_Clip`.
    - Work with both in-database rasters (stored in memory) and out-of-database rasters (linked to GeoTIFFs or COGs on S3).

2. Spatial Statistics and Pruning

    Each Iceberg data file stores spatial metadata — including minimum bounding rectangles (MBRs) for geometry and raster columns.

    This allows the query engine to:
    - Skip irrelevant files that fall outside your spatial filter (spatial pruning).
    - Push down bounding-box filters to the scan layer, reducing data read from storage.
    - Speed up spatial joins by comparing file-level extents before loading data.

3. Spatial Partitioning and Organization

    Spatial transformations (like tiling, grid partitioning, or bounding-box bucketing) can be used to organize data.
    This helps co-locate nearby geometries and tiles, reducing shuffle and improving performance in spatial joins or aggregations.

4. Query Optimization and Pushdown

    WherobotsDB uses Iceberg’s metadata to push down both spatial and temporal filters:
    - Spatial filters (`ST_Intersects`, `ST_Within`) are evaluated at the metadata level.
    - Raster metadata and specific bands can be selectively read (projection pushdown).

In [ ]:
# Transforming data with non-native readers - in slides

In [ ]:
# Wherobots Fundamentals - Constructing Geometries - Furqaan

In [ ]:
# Wherobots Fundamentals - Spatial predicates  - Furqaan

In [ ]:
# Wherobots Fundamentals - Range joins - Furqaan

In [ ]:
# create and manage Havasu (Iceberg) tables for vector and raster data  - Furqaan

# Data validity checks

Two of the most common issues with geospatial data include managing projections or Coordinate Reference Systems (CRS) and ensuring geometries are valid.

- A geometry is invalid if it violates spatial rules like self-intersections, unclosed rings, misaligned holes, or overlapping parts—making it topologically incorrect.
- Spatial files generally contain a Coordinate Reference System or CRS that is defined by a Spatial Reference ID or SRID. This tells us how the data is projected from the round spheroid of the earth onto a flat surface.

To fix these issues and ensure our data is valid and in the correct format we use two approaches:

1. Check the geometries for any invalidities, and if there are attempt to fix them using `ST_IsValid`, `ST_IsValidDetail`, and `ST_MakeValid`
2. Remove or log out any geometries that cannot be fixed
3. Standardize our geometries in a single CRS, in this case [EPSG:4326](https://epsg.io/4326) which renders in a coordinate reference system

## Validating geometries

In [ ]:
# Data validity checks - Matt

## Transforming CRS

In [ ]:
# Handling and transforming CRS - Matt

In [ ]:
# Dataset loading - aka load all datasets to tables - Matt

# Loading datasets into WherobotsDB

In [4]:
prefix = 's3://wherobots-examples/gdea-course-data/raw-data/'
database = 'gde_bronze'

In [5]:
sedona.sql(f'CREATE DATABASE IF NOT EXISTS wherobots.{database}')

DataFrame[]

In [6]:
def check_invalid_geometries(df: DataFrame, geom_col: str = "geom", reason_col: str = "why_invalid") -> int:
    df_with_reason = df.withColumn(reason_col, ST_IsValidReason(col(geom_col)))
    # cache to avoid recomputation if you inspect reasons later
    df_with_reason.cache()
    invalid_count = df_with_reason.filter(~ST_IsValid(col(geom_col))).count()
    print(f"✅ Checked geometries — found {invalid_count} invalid geometries.")
    return invalid_count

def fix_invalid_geometries(df: DataFrame, invalid_count: int, geom_col: str = "geom") -> DataFrame:
    if invalid_count > 1:
        print(f"🔧 Attempting to fix {invalid_count} invalid geometries...")
        return df.withColumn(
            geom_col,
            when(~ST_IsValid(col(geom_col)), ST_MakeValid(col(geom_col))).otherwise(col(geom_col))
        )
    else:
        print("⚡ Only one invalid geometry (or none). Skipping automated fix.")
        return df

# --- driver program ---
def process_geometries(
    df: DataFrame,
    geom_col: str = "geom",
    attempt_fix: bool = True,
    split_on_fail: bool = True
):
    """
    Runs validity check -> optional repair -> optional split.
    Returns either:
      - {"df": corrected_df}  when all geometries valid after repair (or none invalid)
      - {"valid_df": ..., "invalid_df": ...} when some invalid remain and split_on_fail=True
    """
    # 1) Initial check
    invalid_count = check_invalid_geometries(df, geom_col=geom_col)

    if invalid_count == 0:
        print("✅ All geometries are valid.")
        return {"df": df}  # nothing to do

    # 2) Attempt repair (only changes rows that are invalid per your earlier contract)
    if attempt_fix:
        df_fixed = fix_invalid_geometries(df, invalid_count, geom_col=geom_col)
        remaining_invalid_count = df_fixed.filter(~ST_IsValid(col(geom_col))).count()
        print(f"🔎 After fixing, {remaining_invalid_count} invalid geometries remain.")
    
        if remaining_invalid_count == 0:
            print("✅ All geometries are valid after fixing.")
            return {"df": df_fixed}
        elif split_on_fail:
            print("⚠️ Some invalid geometries remain — splitting dataset.")
            valid_df = df_fixed.filter(ST_IsValid(col(geom_col)))
            invalid_df = df_fixed.filter(~ST_IsValid(col(geom_col)))
            print(f"✅ Split complete: {valid_df.count()} valid / {invalid_df.count()} invalid.")
            return {"valid_df": valid_df, "invalid_df": invalid_df}
        else:
            print("⚠️ Some invalid geometries remain, returning best-effort fixed DataFrame.")
            return {"df": df_fixed}
    
    # If no fix attempt, just split if requested
    if split_on_fail:
        print("⚠️ Skipping fix — splitting into valid and invalid.")
        valid_df = df.filter(ST_IsValid(col(geom_col)))
        invalid_df = df.filter(~ST_IsValid(col(geom_col)))
        print(f"✅ Split complete: {valid_df.count()} valid / {invalid_df.count()} invalid.")
        return {"valid_df": valid_df, "invalid_df": invalid_df}
    
    print("⚠️ Invalid geometries found but no fix or split requested. Returning original DataFrame.")
    return {"df": df}

In [7]:
# FEMA Flood Hazard Areas
fld_hazard_area = sedona.read.format('shapefile').load(f'{prefix}' + '53033C_20250330/S_FLD_HAZ_AR.shp')

In [40]:
result = process_geometries(fld_hazard_area, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

25/09/30 18:43:28 WARN CacheManager: Asked to cache already cached data.


✅ Checked geometries — found 15 invalid geometries.
🔧 Attempting to fix 15 invalid geometries...


🔎 After fixing, 0 invalid geometries remain.
✅ All geometries are valid after fixing.


In [41]:
df_final.writeTo(f"wherobots.{database}.fema_flood_zones_bronze").createOrReplace()

In [42]:
# King County Generalized Land Use Data
gen_land_use = sedona.read.format('shapefile').load(f'{prefix}' + 'General_Land_Use_Final_Dataset/General_Land_Use_Final_Dataset.shp')

In [43]:
result = process_geometries(gen_land_use, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 2987 invalid geometries.
🔧 Attempting to fix 2987 invalid geometries...


🔎 After fixing, 0 invalid geometries remain.
✅ All geometries are valid after fixing.


In [44]:
valid_df.writeTo(f"wherobots.{database}.gen_land_use_bronze").createOrReplace()

In [45]:
# King County Sherrif Patrol Districts
sherrif_districts = sedona.read.format('shapefile').load(f'{prefix}' + 'King_County_Sheriff_Patrol_Districts___patrol_districts_area/King_County_Sheriff_Patrol_Districts___patrol_districts_area.shp')

In [46]:
result = process_geometries(sherrif_districts, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


In [47]:
df_final.writeTo(f"wherobots.{database}.sherrif_districts_bronze").createOrReplace()

In [48]:
offense_reports = sedona.read.format('csv').load(f'{prefix}' + 'KCSO_Offense_Reports__2020_to_Present_20250923.csv')

In [49]:
offense_reports.writeTo(f"wherobots.{database}.offense_reports_bronze").createOrReplace()

In [50]:
# King County Bike Lanes
bike_lanes = sedona.read.format('shapefile').load(f'{prefix}' + 'Metro_Transportation_Network_(TNET)_in_King_County_for_Bicycle_Mode___trans_network_bike_line/Metro_Transportation_Network_(TNET)_in_King_County_for_Bicycle_Mode___trans_network_bike_line.shp')

In [51]:
result = process_geometries(bike_lanes, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


In [52]:
df_final.writeTo(f"wherobots.{database}.bike_lanes_bronze").createOrReplace()

In [53]:
# FEMA National Risk Index
fema_nri = sedona.read.format('shapefile').load(f'{prefix}' + 'NRI_Shapefile_CensusTracts/NRI_Shapefile_CensusTracts.shp')

In [56]:
result = process_geometries(fema_nri, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

25/09/30 19:01:47 WARN CacheManager: Asked to cache already cached data.


✅ Checked geometries — found 82 invalid geometries.
🔧 Attempting to fix 82 invalid geometries...


🔎 After fixing, 0 invalid geometries remain.
✅ All geometries are valid after fixing.


In [57]:
df_final.writeTo(f"wherobots.{database}.fema_nri_bronze").createOrReplace()

25/09/30 19:19:03 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed.


In [8]:
# King County School Sites
school_sites = sedona.read.format('shapefile').load(f'{prefix}' + 'School_Sites_in_King_County___schsite_point/School_Sites_in_King_County___schsite_point.shp')

In [9]:
result = process_geometries(school_sites, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


In [10]:
df_final.writeTo(f"wherobots.{database}.school_sites_bronze").createOrReplace()

In [20]:
# Schools Report Card
report_card = sedona.read. \
    format('csv'). \
    load(f'{prefix}' + 'Report_Card_Growth_for_2024-25_20250923.csv')

In [21]:
report_card.writeTo(f"wherobots.{database}.report_card_bronze").createOrReplace()

In [22]:
# Seismic Hazards
seismic_hazards = sedona.read. \
    format('shapefile'). \
    load(f'{prefix}' + 'Seismic_Hazards___seism_area/Seismic_Hazards___seism_area.shp')

In [23]:
result = process_geometries(seismic_hazards, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


25/10/01 02:01:27 WARN CacheManager: Asked to cache already cached data.


In [24]:
df_final.writeTo(f"wherobots.{database}.seismic_hazards_bronze").createOrReplace()

In [25]:
# Census Block Groups
block_groups = sedona.read. \
    format('shapefile'). \
    load(f'{prefix}' + 'tl_2024_53_bg/tl_2024_53_bg.shp')

In [26]:
result = process_geometries(block_groups, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

25/10/01 02:01:54 WARN CacheManager: Asked to cache already cached data.


✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


In [27]:
df_final.writeTo(f"wherobots.{database}.block_groups_bronze").createOrReplace()

In [28]:
# Census CSVs
median_age = sedona.read. \
    format('csv'). \
    load(f'{prefix}' + 'ACSDT5Y2023.B01002_2025-09-19T105233/ACSDT5Y2023.B01002-Data.csv')

median_age.writeTo(f"wherobots.{database}.median_age_bronze").createOrReplace()

total_pop = sedona.read. \
    format('csv'). \
    load(f'{prefix}' + 'ACSDT5Y2023.B01003_2025-09-19T105050/ACSDT5Y2023.B01003-Data.csv')

total_pop.writeTo(f"wherobots.{database}.total_pop_bronze").createOrReplace()

median_income = sedona.read. \
    format('csv'). \
    load(f'{prefix}' + 'ACSDT5Y2023.B19013_2025-09-19T105253/ACSDT5Y2023.B19013-Data.csv')

total_pop.writeTo(f"wherobots.{database}.median_income_bronze").createOrReplace()

In [29]:
# Tranist Routes
transit_routes = sedona.read. \
    format('shapefile'). \
    load(f'{prefix}' + 'Transit_Routes_for_King_County_Metro___transitroute_line/Transit_Routes_for_King_County_Metro___transitroute_line.shp')

In [30]:
result = process_geometries(transit_routes, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


In [31]:
df_final.writeTo(f"wherobots.{database}.transit_routes_bronze").createOrReplace()

In [32]:
# Transit Stops
transit_stops = sedona.read. \
    format('shapefile'). \
    load(f'{prefix}' + 'Transit_Stops_for_King_County_Metro___transitstop_point/Transit_Stops_for_King_County_Metro___transitstop_point.shp')

In [33]:
result = process_geometries(transit_stops, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


In [34]:
df_final.writeTo(f"wherobots.{database}.transit_stops_bronze").createOrReplace()

In [35]:
# Water Bodies
water_bodies = sedona.read. \
    format('shapefile'). \
    load(f'{prefix}' + 'Waterbodies_with_History_and_Jurisdictional_detail___wtrbdy_det_area/Waterbodies_with_History_and_Jurisdictional_detail___wtrbdy_det_area.shp')

In [36]:
result = process_geometries(water_bodies, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 5 invalid geometries.
🔧 Attempting to fix 5 invalid geometries...


🔎 After fixing, 0 invalid geometries remain.
✅ All geometries are valid after fixing.


In [37]:
df_final.writeTo(f"wherobots.{database}.water_bodies_bronze").createOrReplace()

In [39]:
# Wildfire Polygons
wildfires = sedona.read. \
    format('shapefile'). \
    load(f'{prefix}' + 'Wildfires_1878_2019_Polygon_Data/Shapefile/US_Wildfires_1878_2019.shp')

In [40]:
result = process_geometries(wildfires, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 1203 invalid geometries.
🔧 Attempting to fix 1203 invalid geometries...


🔎 After fixing, 0 invalid geometries remain.
✅ All geometries are valid after fixing.


In [41]:
df_final.writeTo(f"wherobots.{database}.wildfires_bronze").createOrReplace()

In [11]:
# Wildfire Rasters

In [12]:
# Elevation

# https://s3.opengeohub.org/global/edtm/gedtm_rf_m_30m_s_20060101_20151231_go_epsg.4326.3855_v20250611.tif	

In [50]:
# Geocoded Schools
schools = sedona.read. \
    format('geojson'). \
    load(f'{prefix}' + 'Washington_State_Public_Schools_GeoCoded.geojson')

In [54]:
schools = schools \
    .withColumn("geometry", expr("geometry")) \
    .withColumn("AYPCode", expr("properties['AYPCode']")) \
    .withColumn("CongressionalDistrict", expr("properties['CongressionalDistrict']")) \
    .withColumn("County", expr("properties['County']")) \
    .withColumn("ESDCode", expr("properties['ESDCode']")) \
    .withColumn("ESDName", expr("properties['ESDName']")) \
    .withColumn("Email", expr("properties['Email']")) \
    .withColumn("GeoCoded_X", expr("properties['GeoCoded_X']")) \
    .withColumn("GeoCoded_Y", expr("properties['GeoCoded_Y']")) \
    .withColumn("GradeCategory", expr("properties['GradeCategory']")) \
    .withColumn("HighestGrade", expr("properties['HighestGrade']")) \
    .withColumn("LEACode", expr("properties['LEACode']")) \
    .withColumn("LEAName", expr("properties['LEAName']")) \
    .withColumn("LegislativeDistrict", expr("properties['LegislativeDistrict']")) \
    .withColumn("LowestGrade", expr("properties['LowestGrade']")) \
    .withColumn("MailingAddress", expr("properties['MailingAddress']")) \
    .withColumn("NCES_X", expr("properties['NCES_X']")) \
    .withColumn("NCES_Y", expr("properties['NCES_Y']")) \
    .withColumn("Phone", expr("properties['Phone']")) \
    .withColumn("Principal", expr("properties['Principal']")) \
    .withColumn("School", expr("properties['School']")) \
    .withColumn("SchoolCategory", expr("properties['SchoolCategory']")) \
    .withColumn("SchoolCode", expr("properties['SchoolCode']")) \
    .withColumn("SingleAddress", expr("properties['SingleAddress']")) \
    .drop("properties").drop("type") \
    .drop("_corrupt_record").drop("type") \
    .drop("type").drop("type")

In [58]:
schools.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- AYPCode: string (nullable = true)
 |-- CongressionalDistrict: string (nullable = true)
 |-- County: string (nullable = true)
 |-- ESDCode: long (nullable = true)
 |-- ESDName: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- GeoCoded_X: double (nullable = true)
 |-- GeoCoded_Y: double (nullable = true)
 |-- GradeCategory: string (nullable = true)
 |-- HighestGrade: string (nullable = true)
 |-- LEACode: long (nullable = true)
 |-- LEAName: string (nullable = true)
 |-- LegislativeDistrict: string (nullable = true)
 |-- LowestGrade: string (nullable = true)
 |-- MailingAddress: string (nullable = true)
 |-- NCES_X: double (nullable = true)
 |-- NCES_Y: double (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Principal: string (nullable = true)
 |-- School: string (nullable = true)
 |-- SchoolCategory: string (nullable = true)
 |-- SchoolCode: long (nullable = true)
 |-- SingleAddress: string (nullable = true

In [55]:
result = process_geometries(schools, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

25/10/01 02:29:06 ERROR TaskSetManager: Task 0 in stage 84.0 failed 4 times; aborting job


Py4JJavaError: An error occurred while calling o494.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 84.0 failed 4 times, most recent failure: Lost task 0.3 in stage 84.0 (TID 1698) (10.1.31.162 executor 8): java.lang.NullPointerException: Cannot invoke "org.apache.spark.unsafe.types.UTF8String.toString()" because the return value of "org.apache.spark.sql.catalyst.InternalRow.getUTF8String(int)" is null
	at org.apache.spark.sql.catalyst.InternalRow.getString(InternalRow.scala:35)
	at org.apache.spark.sql.sedona_sql.io.geojson.GeoJSONUtils$.$anonfun$convertGeoJsonToGeometry$1(GeoJSONUtils.scala:111)
	at org.apache.spark.sql.sedona_sql.io.geojson.GeoJSONUtils$.$anonfun$convertGeoJsonToGeometry$1$adapted(GeoJSONUtils.scala:109)
	at scala.collection.ArrayOps$.foreach$extension(ArrayOps.scala:1328)
	at org.apache.spark.sql.sedona_sql.io.geojson.GeoJSONUtils$.convertGeoJsonToGeometry(GeoJSONUtils.scala:109)
	at org.apache.spark.sql.sedona_sql.io.geo

In [ ]:
df_final.writeTo(f"wherobots.{database}.schools_bronze").createOrReplace()